In [65]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.companies

from pandas.io.json import json_normalize

In [2]:
#Starting consuting time on Mongo DB

one_office = db.companies.find({    #Companies with at least 1 office
    "offices":{
        "$not":{
            "$size":0
        }
    }
},{"name":1, "offices":1})

one_office_df = pd.DataFrame(one_office)
#display(one_office_df)
print(one_office_df.shape)
one_office_df.columns


(13744, 3)


Index(['_id', 'name', 'offices'], dtype='object')

In [123]:
#Filtering with mongo
 
mongodata = db.companies.find( {"$and":[
                                {"offices.latitude": {"$exists": True}},
                                {"offices.latitude": {"$ne": None}},   
                                {"offices.longitude": {"$exists": True}},
                                {"offices.longitude": {"$ne": None}},
                                {"category_code": {"$exists": True}},
                                {"category_code": {"$ne": None}},   
                                {"founded_year": {"$exists": True}}, 
                                {"founded_year": {"$gte": 2000}},
                                {"total_money_raised": {"$exists": True}},
                                {"total_money_raised": {"$not":{"$size":0}}}, 
                                {"$or": [
                                {"total_money_raised": {"$gte": 1000000}},
                                {"category_code":  "design" } ,
                                {"category_code":  "nanotech" } , 
                                {"category_code":  "web" } , 
                                {"category_code":  "software" } , 
                                {"category_code":  "games_video" } , 
                                {"category_code":  "mobile" } , 
                                {"category_code":  "ecommerce" } ,
                                {"category_code":  "advertising" } ,
                                {"category_code":  "enterprise" } ,   
                                {"category_code":  "analytics" } ,   
                                {"category_code":  "photo_video" } ,   
                                {"category_code":  "biotech" } ]} ,
                                      
                                ]
                                },
                                {
                               "_id": 0, "category_code": 1,"founded_year": 1, "name": 1, 
                               "offices.country_code": 1, "offices.latitude": 1, "offices.longitude": 1, 
                                "total_money_raised": 1   
                                }
                            )
    
    
  

In [4]:
mongodata

In [63]:
mydataset = db.companies.find({"$and": 
                               [{"offices.latitude": {"$exists": True}}, 
                                {"offices.latitude": {"$ne": None}},
                                {"offices.longitude": {"$exists": True}}, 
                                {"offices.longitude": {"$ne": None}},
                                {"category_code": {"$exists": True}}, 
                                {"category_code": {"$ne": None}},
                                {"number_of_employees": {"$exists": True}}, 
                                {"number_of_employees": {"$ne": None}},
                                {"founded_year": {"$exists": True}}, 
                                {"founded_year": {"$gte": 2008}}]}, 
                              {"_id": 0, "category_code": 1,"founded_year": 1, "name": 1, 
                               "number_of_employees": 1, "offices.country_code": 1, 
                               "offices.latitude": 1, "offices.longitude": 1, "total_money_raised": 1})

In [108]:
df = pd.DataFrame(mongodata)
display(df.head())

,category_code,founded_year,name,offices,total_money_raised
0,web,2005,Wetpaint,"[{'country_code': 'USA', 'latitude': 47.603122...",$39.8M
1,software,2005,Zoho,"[{'country_code': 'USA', 'latitude': 37.692934...",$0
2,web,2006,Geni,"[{'country_code': 'USA', 'latitude': 34.090368...",$16.5M
3,web,2003,Gizmoz,"[{'country_code': 'USA', 'latitude': 37.48413,...",$18.1M
4,mobile,2005,Helio,"[{'country_code': 'USA', 'latitude': 34.057498...",$0


In [109]:
print(df["offices"][0])
print(df["name"][0])

#def count_offices(data):
#    return {"total_offices": len(data)}

[{'country_code': 'USA', 'latitude': 47.603122, 'longitude': -122.333253}, {'country_code': 'USA', 'latitude': 40.7237306, 'longitude': -73.9964312}]
Wetpaint


In [105]:
#df = df[["offices"]].apply(count_offices, result_type = "expand", axis = 1)

In [110]:
print(df.columns)

Index(['category_code', 'founded_year', 'name', 'offices',
       'total_money_raised'],
      dtype='object')


In [124]:
ydataframe = json_normalize(data = mongodata, record_path = "offices", 
                             meta = ["category_code", "founded_year", "name",
                                    "total_money_raised"])

In [125]:
display(ydataframe.head())
print(ydataframe.shape)
print(ydataframe.columns)

,country_code,latitude,longitude,category_code,founded_year,name,total_money_raised
0,USA,47.603122,-122.333253,web,2005,Wetpaint,$39.8M
1,USA,40.723731,-73.996431,web,2005,Wetpaint,$39.8M
2,USA,37.692934,-121.904945,software,2005,Zoho,$0
3,USA,34.090368,-118.393064,web,2006,Geni,$16.5M
4,USA,37.484130,-122.169472,web,2003,Gizmoz,$18.1M


(4835, 7)
Index(['country_code', 'latitude', 'longitude', 'category_code',
       'founded_year', 'name', 'total_money_raised'],
      dtype='object')


In [133]:
df = ydataframe.reindex(columns=['country_code', 'name', 'founded_year', 'category_code', 'latitude', 'longitude', 'total_money_raised'])

In [134]:
display(df.head())

,country_code,name,founded_year,category_code,latitude,longitude,total_money_raised
0,USA,Wetpaint,2005,web,47.603122,-122.333253,$39.8M
1,USA,Wetpaint,2005,web,40.723731,-73.996431,$39.8M
2,USA,Zoho,2005,software,37.692934,-121.904945,$0
3,USA,Geni,2006,web,34.090368,-118.393064,$16.5M
4,USA,Gizmoz,2003,web,37.484130,-122.169472,$18.1M


In [138]:

df["total_offices"] = df["name"].map(df["name"].value_counts())

In [139]:
display(df.head())

,country_code,name,founded_year,category_code,latitude,longitude,total_money_raised,total_offices
0,USA,Wetpaint,2005,web,47.603122,-122.333253,$39.8M,2
1,USA,Wetpaint,2005,web,40.723731,-73.996431,$39.8M,2
2,USA,Zoho,2005,software,37.692934,-121.904945,$0,1
3,USA,Geni,2006,web,34.090368,-118.393064,$16.5M,1
4,USA,Gizmoz,2003,web,37.484130,-122.169472,$18.1M,1


In [95]:
def get_offices(data):
    data = data["offices"]
    offices = None
    if data["latitude"] and data["longitude"]:
        offices = {
            "type":"Point",
            "coordinates":[data["longitude"], data["latitude"]]
        }
    return {
        "totalOffices": len(data),
        "lat": data["latitude"],
        "lng": data["longitude"],
        "Offices": offices
    }    



In [93]:
df_norm = df[["offices"]].apply(get_offices, result_type = "expand", axis = 1)
print(df_norm.shape)
display(df_norm.head())

KeyError: "None of [Index(['offices'], dtype='object')] are in the [columns]"

""


In [7]:
def getFirstOffice(data):
    data = data['offices']
    #Only create the geoJSON object if all geodata is available
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }
    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "oficina_principal": principal
    }


first_office = one_office_df[["offices"]].apply(getFirstOffice, result_type = "expand", axis = 1)
print(first_office.shape)
display(first_office.head())

(13744, 4)


,lat,lng,oficina_principal,totalOffices
0,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
1,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,...",2.0
2,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
3,NaN,NaN,NaN,1.0
4,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,...",1.0


In [8]:
df_clean = pd.concat([one_office_df,first_office], axis=1)[["name","lat","lng", "oficina_principal","totalOffices"]]

display(df_clean.head())


,name,lat,lng,oficina_principal,totalOffices
0,AdventNet,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
1,Wetpaint,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,...",2.0
2,Zoho,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
3,Omnidrive,NaN,NaN,NaN,1.0
4,Geni,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,...",1.0


In [9]:
def drop_null(df):
    return df.dropna(axis=0)

In [10]:
df_cleaned = df_clean.dropna()

In [11]:
df_cleaned.head()
df_cleaned.shape


(9618, 5)